## Using a simple RNN for machine translation
This project is still in progress.

I'm busy doing the Udacity Natural Language Processing Nanodegree and I'm busy trying build an effective English to French translation model.

In [ ]:
# TODO
# Import translation data

In [2]:
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, LSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


In [1]:
# Preprosessing 
def tokenize(x):
    text_tokenizer = Tokenizer()
    text_tokenizer.fit_on_texts(x)
    text_tokenized = text_tokenizer.texts_to_sequences(x)
    return text_tokenized, text_tokenizer

def pad(x, length=None):
    if length == None:
        length = max([len(sentance) for sentance in x])
    x = np.array(x)
    x_padded = pad_sequences(x, maxlen=length, dtype='int32', padding='post')
    return x_padded

def preprocess(x, y):
    preprocess_x, x_token = tokenize(x)
    preprocess_y, y_token = tokenize(y)
    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
    return preprocess_x, preprocess_y, x_token, y_token